In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver import ChromeOptions
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.action_chains import ActionChains

import time
import pandas as pd
import numpy as np

pd.options.mode.chained_assignment = None

########### HELPER FUNCTIONS #######################

def evaluate_order(VALUE: int, CASE: str, dX: int = 5):
        
    operator = CASE.split(' ')[1]
            
    if (VALUE > 95) and (VALUE < 101):
        dX = (100 - VALUE) - 1
                
    if operator == '<':
        return VALUE - dX
    if operator == '>':
        return VALUE + dX
    return VALUE
    

######################################################



class TestLoan:
    
    loan_purpose, loan_type = '',''
    
    def __init__(self,LOAN_PURPOSE: str, LOAN_TYPE: str):
        
        self.loan_purpose = LOAN_PURPOSE
        self.loan_type = LOAN_TYPE
        
    def set_loan_parameters(self,OCCUPANCY: str, CHANNEL: str, STATE: str, PROPERTY_TYPE: str, PPP: str, INVESTOR: str, PRICING_VARIABLES: str, ID: str):
        
        self.occupancy = OCCUPANCY
        self.channel = CHANNEL
        self.state = STATE
        self.property_type = PROPERTY_TYPE
        self.ppp = PPP
        self.investor = INVESTOR
        self.pricing_variables = PRICING_VARIABLES
        self.id = ID
        
    def set_vesta_url(self,URL: str):
        
        self.vesta_url = URL
        
    def get_vesta_url(self):
        
        return self.vesta_url
    
            
    def parse_pricing_variables(self):
        
        if self.loan_type == 'Conv':
        
            dti_string = (self.pricing_variables.split('L')[0])[:-2]
            ltv_string = ('L' + (self.pricing_variables.split('L')[1]).split('C')[0])[:-2]
            fico_string = 'C' + self.pricing_variables.split('C')[1]
            
            dti_cases = dti_string.split(',')
            dti_cases[1] = 'DTI' + dti_cases[1]

            ltv_cases = ltv_string.split(',',3)
            ltv_cases[1] = 'LTV' + ltv_cases[1]
            ltv_cases[2] = 'LTV' + ltv_cases[2]

            fico_cases = fico_string.split(',')
            fico_cases[1] = 'Credit_Score' + fico_cases[1]
            
            cases = [0] * (len(dti_cases) * len(ltv_cases) * len(fico_cases))
            
            idx = 0
            for i in range(len(dti_cases)):
                for j in range(len(ltv_cases)):
                    for k in range(len(fico_cases)):
                        
                        dti_ijk = evaluate_order(VALUE=int(dti_cases[i][-2:]),CASE=dti_cases[i])
                        ltv_ijk = evaluate_order(VALUE=int(ltv_cases[j][-2:]),CASE=ltv_cases[j])
                        fico_ijk = evaluate_order(VALUE=int(fico_cases[k][-3:]),CASE=fico_cases[k])
                        
                        cases[idx] = (dti_ijk,ltv_ijk,fico_ijk,dti_cases[i] + ' ' + ltv_cases[j] + ' ' + fico_cases[k])
                        idx += 1
                        
            
        if self.loan_type == 'BS':
            
            Amonth_string = self.pricing_variables.split(',')[0]
            Bmonth_string = (self.pricing_variables.split(',')[1])[1:]
            
            caseA = ( int(Amonth_string[0:2]), Amonth_string )
            caseB = ( int(Bmonth_string[0:2]), Bmonth_string)
            
            cases = [caseA, caseB]
            
        
        if self.loan_type == 'DSCR':
            cases = [(0,0,0,' ')] * 3
            
        
        return cases

class Web_Pilot:
    
    browser, service = None, None
    actions = None
    
    investor_dict = {
        
        "PennyMac": "Pennymac",
        "UWM": "United Wholesale Mortgage",
        "Deephaven": "Deephaven",
        "NewRez": "Newrez",
        "Verus": "Verus",
        "Arc": "Arc Home Loans",
        "Mr. Cooper": "Mr. Cooper"
        
    }
    
    
    # initialize webdriver with the path to chromedriver.exe
    def __init__(self):
                
        self.opts = ChromeOptions()
        self.opts.add_argument("--no-sandbox")
        
        self.browser = webdriver.Chrome(options=self.opts)
        self.actions = ActionChains(self.browser)
        
    def wait(self,dt: float=1):
        time.sleep(dt)
        
    def open_page(self,URL: str):
        self.browser.get(URL)
        
    def close_browser(self):
        self.browser.close()
    
    def add_input(self, BY: By, VALUE: str, TEXT: str):
        field = self.browser.find_element(by=BY,value=VALUE)
        field.send_keys(TEXT)
        self.wait(0.5)
            
    def enter_input(self,BY: By, VALUE: str, GLOBAL: bool = False):
        
        if GLOBAL:
            
            field = self.browser.find_element(by=BY,value=VALUE)
            self.actions.move_to_element(field)
            self.actions.click()
            self.actions.perform()
            
        else:
                
            field = self.browser.find_element(by=BY,value=VALUE)
            field.send_keys(Keys.ENTER)
            self.wait(0.5)
    
    def moveDown_input(self,BY: By, VALUE: str):
        field = self.browser.find_element(by=BY,value=VALUE)
        field.send_keys(Keys.DOWN)
        self.wait(0.5)
        
    def select_input(self, BY: By, VALUE: str, TEXT: str):
        field = Select(self.browser.find_element(by=BY,value=VALUE))
        field.select_by_index(1)
        self.wait(0.5)
        
    def clear_input(self, BY: By, VALUE: str, KEYS: int=1):
        field = self.browser.find_element(by=BY,value=VALUE)
        
        for i in range(KEYS):
            field.send_keys(Keys.BACK_SPACE)
            self.wait(0.5)
        
        
    def click_button(self, BY: By, VALUE: str):
        button = self.browser.find_element(by=BY,value=VALUE)
        button.click()
        self.wait(0.5)
        
    def select_WebElements(self, BY: By, VALUE: str):
        return self.browser.find_elements(by=BY,value=VALUE)
    
    def select_WebElement(self,BY: By, VALUE: str):
        return self.browser.find_element(by=BY,value=VALUE)
    
    def getText_WebElement(self,BY: By, VALUE: str):
        return self.select_WebElement(BY=BY,VALUE=VALUE).text
    
    
        

    
class Price_Engine_Pilot(Web_Pilot):
    browser, service = None, None
    username, password = '',''
    actions = None
    
    def __init__(self,USERNAME: str, PASSWORD: str):
        self.username = USERNAME
        self.password = PASSWORD
        
        Web_Pilot.__init__(self)
        
    
    def login_vesta(self):
        
        # Login using Gmail
        self.click_button(BY=By.CLASS_NAME,VALUE="_ProviderLoginButton-module_gsi-material-button_I1g3crH")
        
        # Enter Gmail and click next
        self.add_input(BY=By.CLASS_NAME, VALUE='whsOnd',TEXT=self.username)
        self.click_button(BY=By.CLASS_NAME,VALUE="VfPpkd-LgbsSe-OWXEXe-k8QpJ")
        
        self.wait(2)
        
        # Enter password and click next
        self.add_input(BY=By.CLASS_NAME,VALUE='whsOnd',TEXT=self.password)
        self.click_button(BY=By.CLASS_NAME,VALUE='VfPpkd-LgbsSe-OWXEXe-k8QpJ')
        
        # User input once G-mail 2-factor approved
        input("Verify Credentials and Press Enter to Continue...")
        self.wait(2)
        
        # Open pricing engine
        self.click_button(BY=By.XPATH,VALUE='/html/body/div[1]/div[1]/div[2]/c-wiz/div/div[3]/div/div/div[2]/div/div/button')
        self.wait(10)
        
    def pull_credit(self):
        
        # toggle 'Pull credit' and click it
        self.click_button(BY=By.XPATH,VALUE='/html/body/div[1]/div/div[2]/div[1]/div/div/div/div[1]/div[2]/div[1]/div[3]/div/button')
        self.wait(2.0)
        self.enter_input(BY=By.XPATH,VALUE='/html/body/div[4]/div/div/div/div/div/div[1]/div/div[1]/div',GLOBAL=True)
        self.wait(2.0)
        
        # move cursor to 'Pull new credit report' and click it
        self.enter_input(BY=By.XPATH,VALUE='/html/body/div[5]/div/div[2]/div/div[2]/div/div/div[2]/div/div/div/div[2]/div[1]/label/span[1]/input',GLOBAL=True)
        self.wait(2.0)
        
        # move cursor to 'Next' and click it
        self.enter_input(BY=By.XPATH,VALUE='/html/body/div[5]/div/div[2]/div/div[2]/div/div/div[3]/button',GLOBAL=True)
        self.wait(2.0)
        
        # move cursor to 'Pull Credit' and click it
        self.enter_input(BY=By.XPATH,VALUE='/html/body/div[5]/div/div[2]/div/div[2]/div/div/div[3]/button',GLOBAL=True)
        self.wait(10)
        
        # press close and exit
        self.enter_input(BY=By.XPATH,VALUE='/html/body/div[5]/div/div[2]/div/div[2]/div/div/div[4]/button',GLOBAL=True)
        self.wait(1.0)
        
        
    def pull_fees(self):
        
        # toggle 'Pull fees' and click it
        self.click_button(BY=By.XPATH,VALUE='/html/body/div[1]/div/div[2]/div[1]/div/div/div/div[1]/div[2]/div[1]/div[3]/div/button')
        self.wait(2.0)
        self.enter_input(BY=By.XPATH,VALUE='/html/body/div[4]/div/div/div/div/div/div[1]/div/div[3]/div',GLOBAL=True)
        self.wait(10.0)
        
        # select 'Beeline Title'
        input_text= 'Beeline Title'
        self.add_input(BY=By.XPATH,VALUE='/html/body/div[4]/div[1]/div/div/div/div/div[2]/div/div[2]/div/div/div[2]/div[1]/div/div[1]/input',TEXT=input_text)
        self.wait(3.0)
        self.enter_input(BY=By.XPATH,VALUE='/html/body/div[4]/div[1]/div/div/div/div/div[2]/div/div[2]/div/div/div[2]/div[1]/div/div[1]/input')
        self.wait(3.0)
        
        # click 'Next'
        self.click_button(BY=By.XPATH,VALUE='/html/body/div[4]/div/div/div/div/div/div[3]/button[2]')
        self.wait(2.0)
        
        # click 'Pull fees'
        self.click_button(BY=By.XPATH,VALUE='/html/body/div[4]/div/div/div/div/div/div[3]/button[2]')
        self.wait(10.0)
        
        # click 'Confirm'
        self.click_button(BY=By.XPATH,VALUE='/html/body/div[4]/div/div/div/div/div/div[3]/button[2]')
        self.wait(10.0)



    def fetch_quotes(self):
        
        try: 
            
            # view top products
            self.click_button(BY=By.XPATH,VALUE='//*[@id="_FullScreenModal-module_modal_xmLzJ6A"]/div/div[2]/div[2]/div/div[1]/div/div/div/button')
            self.wait(2)
            
            # get quote WebElements
            quotes = self.select_WebElements(BY=By.XPATH,VALUE='//*[@id="_FullScreenModal-module_modal_xmLzJ6A"]/div/div[2]/div[2]/div/div[1]/div/div[2]/div/div/div[2]/div')
            N_quotes = len(quotes)
            self.wait(2)
            
            # construct dataframe to store quotes
            quotes_df = pd.DataFrame( columns =['Rate','Price','Investor'],index=range(N_quotes))
            quotes_df['Expired'] = False
            
            for i in range(N_quotes):
                
                # get the rate
                rate_PATH = '//*[@id="_FullScreenModal-module_modal_xmLzJ6A"]/div/div[2]/div[2]/div/div[1]/div/div[2]/div/div/div[2]/div[' + str(i+1) + ']/div/div[1]/div/div/div/div/div[2]/div'
                rate = float(self.getText_WebElement(BY=By.XPATH,VALUE=rate_PATH)[:-1])
                
                # get the price
                price_PATH = '//*[@id="_FullScreenModal-module_modal_xmLzJ6A"]/div/div[2]/div[2]/div/div[1]/div/div[2]/div/div/div[2]/div[' + str(i+1) + ']/div/div[3]/div/div/div/div/div[2]/div'
                price = float(self.getText_WebElement(BY=By.XPATH,VALUE=price_PATH))
                
                # get the investor
                investor_PATH  = '//*[@id="_FullScreenModal-module_modal_xmLzJ6A"]/div/div[2]/div[2]/div/div[1]/div/div[2]/div/div/div[2]/div[' + str(i+1) + ']/div/div[6]/div/div[1]/div/div/div[2]/div'
                investor = self.getText_WebElement(BY=By.XPATH,VALUE=investor_PATH).split('-')[0][:-1]
                
                expirationTag = False
                
                # check if product has expired
                try:
                    expirationTag_PATH = '//*[@id="_FullScreenModal-module_modal_xmLzJ6A"]/div/div[2]/div[2]/div/div[1]/div/div[2]/div/div/div[2]/div[' + str(i+1) + ']/div/div[6]/div/div[2]/div/div/div/div[2]/div'
                    expirationTag = self.getText_WebElement(BY=By.XPATH,VALUE=expirationTag_PATH)
                    expirationTag = True
                
                except NoSuchElementException:
                    pass
                
                # append metadate to quotes dataframe
                quotes_df.at[i,'Rate'] = rate
                quotes_df.at[i,'Price'] = price
                quotes_df.at[i,'Investor'] = investor
                quotes_df.at[i,'Expired'] = expirationTag
                
            
            return quotes_df
                
        except NoSuchElementException:
            pass
        
        return None
        
    
    
    def execute(self):
        
        # click 'View Mortgage Products'
        self.click_button(BY=By.XPATH,VALUE='/html/body/div[4]/div[2]/div/div[1]/div[2]/button')
        self.wait(15)
        
        return self.fetch_quotes()
        
        
    def run_pricing(self, TEST_LOAN: TestLoan, INPUT_DF: pd.DataFrame, OUTPUT_DF: pd.DataFrame, INPUT_NAME: str, OUTPUT_NAME: str, IDX: int):
        
        
        if TEST_LOAN.loan_purpose not in ['P&S','Refi','Cashout','TBD']:
            raise ValueError("Invalid Input for 'Loan Purpose.'")
        
        print("Pricing iteration #" + str(IDX))
        print(TEST_LOAN.loan_purpose)
        print(TEST_LOAN.loan_type)
        print(TEST_LOAN.state)
        print(TEST_LOAN.property_type)
        print()
        
            
           
        # get pricing variables string and return all possible cases to test 
        PARSED_pricing_variables = TEST_LOAN.parse_pricing_variables()
    
        # number of cases
        N_CASES_variables = len(PARSED_pricing_variables)
        i = 0
        
        while i < N_CASES_variables:
            
            
            if TEST_LOAN.loan_type != 'DSCR':
                
                self.wait(40)
                    
                # check if 'Hide (5 mins)' pop-up is enabled
                try:
                        
                    self.click_button(BY=By.XPATH,VALUE='/html/body/div[1]/div/div[2]/div[1]/div/div/div/div[1]/div[2]/div[1]/div[3]/div/button')
                    self.wait(2.0)
                        
                except ElementClickInterceptedException:
                        
                    self.click_button(BY=By.XPATH,VALUE='/html/body/div[2]/div/div[2]/button')
                    self.wait(2.0)
                    self.click_button(BY=By.XPATH,VALUE='/html/body/div[1]/div/div[2]/div[1]/div/div/div/div[1]/div[2]/div[1]/div[3]/div/button')
                    self.wait(2.0)
                
                # open pricing engine        
                self.enter_input(BY=By.XPATH,VALUE='/html/body/div[4]/div/div/div/div/div/div[1]/div/div[2]/div',GLOBAL=True)
                self.wait(3.0)
        
        
            if TEST_LOAN.loan_type == 'Conv':
                        
                # get DTI, LTV, FICO for specific case of specified pricing variables
                DTI_i, LTV_i, FICO_i = PARSED_pricing_variables[i][0], PARSED_pricing_variables[i][1], PARSED_pricing_variables[i][2]
                pricing_variables_i = PARSED_pricing_variables[i][3]
                                
                                
                # set Debt-to-Income ratio
                DTI_PATH = '/html/body/div[4]/div[2]/div/div[1]/div[1]/div[1]/div[2]/div/div[8]/div/div/div/div/div[2]/div[1]/div/input'
                self.clear_input(BY=By.XPATH,VALUE=DTI_PATH,KEYS=8)
                self.add_input(BY=By.XPATH,VALUE=DTI_PATH,TEXT=DTI_i)
                self.wait(0.5)
                
                # set FICO Credit Score
                FICO_PATH = '/html/body/div[4]/div[2]/div/div[1]/div[1]/div[1]/div[2]/div/div[12]/div/div/div/div/div[2]/div/div/input'
                self.clear_input(BY=By.XPATH,VALUE=FICO_PATH,KEYS=4)
                self.add_input(BY=By.XPATH,VALUE=FICO_PATH,TEXT=FICO_i)
                self.wait(0.5)
                
                # calculate new with given LTV ratio
                property_value = 400000
                loan_amount = int(round(LTV_i * property_value * 0.01,0))
                
                # set loan amount
                loan_amount_PATH = '/html/body/div[4]/div[2]/div/div[1]/div[1]/div[1]/div[2]/div/div[2]/div/div/div/div/div[2]/div[2]/div/input'
                self.clear_input(BY=By.XPATH,VALUE=loan_amount_PATH,KEYS=7)
                self.add_input(BY=By.XPATH,VALUE=loan_amount_PATH,TEXT=loan_amount)
                self.wait(0.5)
                
                # set purchase price
                purchase_price_PATH = '/html/body/div[4]/div[2]/div/div[1]/div[1]/div[1]/div[2]/div/div[3]/div/div/div/div/div[2]/div[2]/div/input'
                self.clear_input(BY=By.XPATH,VALUE=purchase_price_PATH,KEYS=7)
                self.add_input(BY=By.XPATH,VALUE=purchase_price_PATH,TEXT=property_value)
                self.wait(0.25)
        
                # set property value
                property_value_PATH = '/html/body/div[4]/div[2]/div/div[1]/div[1]/div[1]/div[2]/div/div[4]/div/div[2]/div[2]/div/input'
                self.clear_input(BY=By.XPATH,VALUE=property_value_PATH,KEYS=7)
                self.add_input(BY=By.XPATH,VALUE=property_value_PATH,TEXT=property_value)
                self.wait(0.25)
                
                # set months of reserves
                reserves = 2
                reserves_PATH = '/html/body/div[4]/div[2]/div/div[1]/div[1]/div[1]/div[2]/div/div[13]/div/div/div/div/div[2]/div/div/input'
                self.add_input(BY=By.XPATH,VALUE=reserves_PATH,TEXT=reserves)
                self.wait(0.5)
                
                # set number of units
                units_PATH = '/html/body/div[4]/div[2]/div/div[1]/div[1]/div[2]/div[2]/div/div[2]/div[1]/div[2]/div/div/input'
                self.clear_input(BY=By.XPATH,VALUE=units_PATH,KEYS=1)
                self.add_input(BY=By.XPATH,VALUE=units_PATH,TEXT=1)
                self.wait(0.5)
                
                # adjust number of units if 'Multi Family' property
                if TEST_LOAN.property_type == 'Multi Family':
    
                    self.clear_input(BY=By.XPATH,VALUE=units_PATH,KEYS=1)
                    self.add_input(BY=By.XPATH,VALUE=units_PATH,TEXT=3)
                    self.wait(0.5)
                    
                # select 'US Citizen
                input_text= 'US Citizen'
                self.add_input(BY=By.XPATH,VALUE='/html/body/div[4]/div[2]/div/div[1]/div[1]/div[3]/div[2]/div/div[2]/div/div/div/div/div[1]/div[2]/div[1]/div/div[1]/input',TEXT=input_text)
                self.wait(3.0)
                self.enter_input(BY=By.XPATH,VALUE='/html/body/div[4]/div[2]/div/div[1]/div[1]/div[3]/div[2]/div/div[2]/div/div/div/div/div[1]/div[2]/div[1]/div/div[1]/input')
                self.wait(3.0)
                
                
            if TEST_LOAN.loan_type == 'BS':
                
                # get month for term of bank statement loan and pricing variables string
                month_i, pricing_variables_i = PARSED_pricing_variables[i][0], PARSED_pricing_variables[i][1]
                                
                # set income doc type
                doc_type = 'BusinessBankStmt24Mos'
                
                if month_i == 12:
                    doc_type = 'BusinessBankStmt12Mos'   
                
                self.wait(2.0)
                self.add_input(BY=By.XPATH,VALUE='/html/body/div[4]/div[2]/div/div[1]/div[1]/div[1]/div[2]/div/div[8]/div/div/div/div/div/div[2]/div[1]/div/div[1]/input',TEXT=doc_type)
                self.wait(3.0)
                self.enter_input(BY=By.XPATH,VALUE='/html/body/div[4]/div[2]/div/div[1]/div[1]/div[1]/div[2]/div/div[8]/div/div/div/div/div/div[2]/div[1]/div/div[1]/input')
                self.wait(3.0)
                          
                # set months of reserves
                reserves = 2
                reserves_PATH = '/html/body/div[4]/div[2]/div/div[1]/div[1]/div[1]/div[2]/div/div[14]/div/div/div/div/div[2]/div/div/input'
                self.add_input(BY=By.XPATH,VALUE=reserves_PATH,TEXT=reserves)
                self.wait(0.5)
                
                # set number of units
                units_PATH = '/html/body/div[4]/div[2]/div/div[1]/div[1]/div[2]/div[2]/div/div[2]/div/div[2]/div/div/input'
                self.clear_input(BY=By.XPATH,VALUE=units_PATH,KEYS=1)
                self.add_input(BY=By.XPATH,VALUE=units_PATH,TEXT=1)
                self.wait(0.5)
                
                # adjust number of units if 'Multi Family' property
                if TEST_LOAN.property_type == 'Multi Family':
    
                    self.clear_input(BY=By.XPATH,VALUE=units_PATH,KEYS=1)
                    self.add_input(BY=By.XPATH,VALUE=units_PATH,TEXT=3)
                    self.wait(0.5)
                    
                # select 'US Citizen
                input_text= 'US Citizen'
                self.add_input(BY=By.XPATH,VALUE='/html/body/div[4]/div[2]/div/div[1]/div[1]/div[3]/div[2]/div/div[2]/div/div/div/div/div/div[2]/div[1]/div/div[1]/input',TEXT=input_text)
                self.wait(3.0)
                self.enter_input(BY=By.XPATH,VALUE='/html/body/div[4]/div[2]/div/div[1]/div[1]/div[3]/div[2]/div/div[2]/div/div/div/div/div/div[2]/div[1]/div/div[1]/input')
                self.wait(3.0)
                
                
                
            if TEST_LOAN.loan_type == 'DSCR':
                i += 1
                continue
            
            # run pricing engine and obtain dataframe of test results
            quotes_df = self.execute()
            ############ IMPLEMENT ERROR CATCHING IN PRICING ENGINE #########################
            optimal_quotes_df = self.select_optimal_quotes(QUOTES_DF=quotes_df)
            par_values_test_results_df = self.par_values_test(OPTIMAL_QUOTES_DF=optimal_quotes_df)
            
            
            OUTPUT_DF.at[IDX,'ID'] = TEST_LOAN.id
            OUTPUT_DF.at[IDX,'Loan Purpose'] = TEST_LOAN.loan_purpose
            OUTPUT_DF.at[IDX,'Occupancy'] = TEST_LOAN.occupancy
            OUTPUT_DF.at[IDX,'Channel'] = TEST_LOAN.channel
            OUTPUT_DF.at[IDX,'State'] = TEST_LOAN.state
            OUTPUT_DF.at[IDX,'Property type'] = TEST_LOAN.property_type
            OUTPUT_DF.at[IDX,'Loan Type'] = TEST_LOAN.loan_type
            OUTPUT_DF.at[IDX,'PPP'] = TEST_LOAN.ppp
            OUTPUT_DF.at[IDX,'Vesta Loan'] = TEST_LOAN.get_vesta_url()
            
            if (type(par_values_test_results_df) is not type(None)) and (par_values_test_results_df.shape[0] > 0):
                
                for j in range(len(par_values_test_results_df)):
                    
                    OUTPUT_DF.at[IDX,'ID'] = TEST_LOAN.id
                    OUTPUT_DF.at[IDX,'Loan Purpose'] = TEST_LOAN.loan_purpose
                    OUTPUT_DF.at[IDX,'Occupancy'] = TEST_LOAN.occupancy
                    OUTPUT_DF.at[IDX,'Channel'] = TEST_LOAN.channel
                    OUTPUT_DF.at[IDX,'State'] = TEST_LOAN.state
                    OUTPUT_DF.at[IDX,'Property type'] = TEST_LOAN.property_type
                    OUTPUT_DF.at[IDX,'Loan Type'] = TEST_LOAN.loan_type
                    OUTPUT_DF.at[IDX,'PPP'] = TEST_LOAN.ppp
                    OUTPUT_DF.at[IDX,'Vesta Loan'] = TEST_LOAN.get_vesta_url()
                    
                    OUTPUT_DF.at[IDX,'Rate'] = par_values_test_results_df.iat[j,0]
                    OUTPUT_DF.at[IDX,'Price'] = par_values_test_results_df.iat[j,1]
                    OUTPUT_DF.at[IDX,'Investor'] = par_values_test_results_df.iat[j,2]
                    OUTPUT_DF.at[IDX,'Pass/Fail'] = par_values_test_results_df.iat[j,3]
                    OUTPUT_DF.at[IDX,'Pricing Variables'] = pricing_variables_i
                    
                    # move back to outside once other loan_types implemented
                    IDX += 1
                    
            
            else:
                
                for j, investor in zip(list(range(len(self.investor_dict))),list(self.investor_dict.keys())):
                    
                    OUTPUT_DF.at[IDX,'ID'] = TEST_LOAN.id
                    OUTPUT_DF.at[IDX,'Loan Purpose'] = TEST_LOAN.loan_purpose
                    OUTPUT_DF.at[IDX,'Occupancy'] = TEST_LOAN.occupancy
                    OUTPUT_DF.at[IDX,'Channel'] = TEST_LOAN.channel
                    OUTPUT_DF.at[IDX,'State'] = TEST_LOAN.state
                    OUTPUT_DF.at[IDX,'Property type'] = TEST_LOAN.property_type
                    OUTPUT_DF.at[IDX,'Loan Type'] = TEST_LOAN.loan_type
                    OUTPUT_DF.at[IDX,'PPP'] = TEST_LOAN.ppp
                    OUTPUT_DF.at[IDX,'Vesta Loan'] = TEST_LOAN.get_vesta_url()
                                        
                    OUTPUT_DF.at[IDX,'Rate'] = 0
                    OUTPUT_DF.at[IDX,'Price'] = 0
                    OUTPUT_DF.at[IDX,'Investor'] = investor
                    OUTPUT_DF.at[IDX,'Pass/Fail'] = 'Fail'
                    OUTPUT_DF.at[IDX,'Pricing Variables'] = pricing_variables_i
                    
                    # move back to outside once other loan_types implemented
                    IDX += 1
                    
            # append new row
            OUTPUT_DF = pd.concat([OUTPUT_DF,OUTPUT_DF.tail(1)]).reset_index(drop=True)
                
            # re-open loan in vesta
            self.open_page(TEST_LOAN.get_vesta_url())
            self.wait(10)
            
            print("Case iteration #" + str(i))
            print(pricing_variables_i)
            print()
            print(OUTPUT_DF.tail(3))
            print()
            print()
            
            i += 1
        
        # update output dataframe
        self.wait(10)
        OUTPUT_DF.to_excel(OUTPUT_NAME)
        
        
        # remove used rows from input dataframe
        INPUT_DF = INPUT_DF.tail(INPUT_DF.shape[0] - 1)
        INPUT_DF.to_excel(INPUT_NAME)
        return IDX
    
         
         
         
            
    def select_optimal_quotes(self, QUOTES_DF: pd.DataFrame):
        
        if (type(QUOTES_DF) is type(None)) or (QUOTES_DF.shape[0] == 0):
            return None
        
        copy_df = QUOTES_DF.copy()
        
        # compute residuals of loan values with respect to par value, i.e. '100'
        copy_df['Residuals'] = (copy_df['Price'] - 100).abs()
        copy_df['Residuals'] = pd.to_numeric(copy_df['Residuals'])
        
        # create dataframe for best rates for all investors
        N_investors = len(self.investor_dict)
        optimal_quotes_df = pd.DataFrame( columns =['Rate','Price','Investor'],index=range(N_investors) )
        optimal_quotes_df['Price'] = 0.0
        i = 0
        
        
        for i, investor in zip(list(range(N_investors)), list(self.investor_dict.keys())):
            
            new_df = copy_df[ (copy_df['Investor'] == self.investor_dict[investor]) & (copy_df['Expired'] == False)]
            
            if new_df.shape[0] > 0:
                
                # get index of loan product with smallest residuals
                idx_min = new_df['Residuals'].argmin()
                
                # append to dataframe
                optimal_quotes_df.iat[i,0] = new_df.iat[idx_min,0]
                optimal_quotes_df.iat[i,1] = new_df.iat[idx_min,1]
                optimal_quotes_df.iat[i,2] = new_df.iat[idx_min,2]
        
            
        return optimal_quotes_df
    
    
    def par_values_test(self, OPTIMAL_QUOTES_DF: pd.DataFrame, LOWER_BOUND: float = 97.0, UPPER_BOUND: float = 103.0):
        
        if (type(OPTIMAL_QUOTES_DF) is type(None)) or (OPTIMAL_QUOTES_DF.shape[0] == 0):
            return None
        
        # construct new dataframe for test results
        par_value_results_df = OPTIMAL_QUOTES_DF.copy()
        par_value_results_df['Pass/Fail'] = 'Fail'
        
        # conduct 'Par-Value' test across all investors
        for i in range(len(par_value_results_df)):
        
            optimal_price = pd.to_numeric(float(OPTIMAL_QUOTES_DF.iat[i,1]))
        
            if (optimal_price >= LOWER_BOUND) and (optimal_price <= UPPER_BOUND):
                
                par_value_results_df.iat[i,3] = 'Pass'
            
        
        
        return par_value_results_df
                    
        
        
        

In [6]:
pilot = Price_Engine_Pilot(USERNAME='mliuzza@makeabeeline.com',PASSWORD='retSEC201066$')

INPUT_SHEET = "OB_Test_Loans_Vesta INPUT.xlsx"
OUTPUT_SHEET = 'OB_Test_Loans_Vesta OUTPUT.xlsx'

# read excel sheet containing mortgage pricing parameters
testLoans_df = pd.read_excel(INPUT_SHEET)

# define output excel sheet
output_df = testLoans_df.head(1)

# add auxiliary columns
output_df['Pass/Fail'] = 'N/A'
output_df['Rate'] = 0
output_df['Price'] = 0

# get lists of all relevant testing parameters
LIST_loan_purpose = (testLoans_df['Loan Purpose'].drop_duplicates()).tolist()
LIST_occupancy = (testLoans_df['Occupancy'].drop_duplicates()).tolist()
LIST_channel = (testLoans_df['Channel'].drop_duplicates()).tolist()
LIST_property_type = (testLoans_df['Property type'].drop_duplicates()).tolist()
LIST_loan_type = (testLoans_df['Loan Type'].drop_duplicates()).tolist()
LIST_ppp = (testLoans_df['PPP'].drop_duplicates()).tolist()
LIST_investor = (testLoans_df['Investor'].drop_duplicates()).tolist()
LIST_id = (testLoans_df['Investor'].drop_duplicates()).tolist()


# open Vesta with first test loan
pilot.open_page(testLoans_df.at[0,'Vesta Loan'])
pilot.wait(3)
pilot.login_vesta()

# define loop variables
N_cases = testLoans_df.shape[0]
idx_input = 0
idx_output = output_df.shape[0] - 1

while idx_input < N_cases:
    
    # create new instance of LoanType for pricing test
    test_loan = TestLoan(LOAN_PURPOSE=testLoans_df.at[idx_input,'Loan Purpose'],LOAN_TYPE=testLoans_df.at[idx_input,'Loan Type'])
    test_loan.set_loan_parameters(OCCUPANCY=testLoans_df.at[idx_input,'Occupancy'],
                                    CHANNEL=testLoans_df.at[idx_input,'Channel'],
                                    STATE=testLoans_df.at[idx_input,'State'],
                                    PROPERTY_TYPE=testLoans_df.at[idx_input,'Property type'],
                                    PPP=testLoans_df.at[idx_input,'PPP'],
                                    INVESTOR=testLoans_df.at[idx_input,'Investor'],
                                    PRICING_VARIABLES = testLoans_df.at[idx_input,'Pricing Variables'],
                                    ID = testLoans_df.at[idx_input,'ID'])
    
    # set URL of test loan in vesta
    test_loan.set_vesta_url(URL=testLoans_df.at[idx_input,'Vesta Loan'])
    
    # get URL of test loan in vesta and open page
    pilot.open_page(URL=test_loan.get_vesta_url())
    pilot.wait(10)
    pilot.pull_credit()
    pilot.pull_fees()
    idx_output = pilot.run_pricing(TEST_LOAN=test_loan,INPUT_DF=testLoans_df,OUTPUT_DF=output_df,INPUT_NAME=INPUT_SHEET,OUTPUT_NAME=OUTPUT_SHEET,IDX=idx_output)
    pilot.wait(10)
    
    output_df = pd.read_excel(OUTPUT_SHEET)
    idx_input += 1


# close browser
pilot.wait(20)
pilot.close_browser()

Pricing iteration #0
P&S
Conv
Florida
Single Family

Case iteration #0
DTI < 45 LTV < 80 Credit_Score < 620

      ID Loan Purpose Occupancy Channel    State  Property type Loan Type   
5  329.0          P&S   Primary  Retail  Florida  Single Family      Conv  \
6  329.0          P&S   Primary  Retail  Florida  Single Family      Conv   
7  329.0          P&S   Primary  Retail  Florida  Single Family      Conv   

   PPP        Investor                     Pricing Variables   
5  NaN  Arc Home Loans  DTI < 45 LTV < 80 Credit_Score < 620  \
6  NaN      Mr. Cooper  DTI < 45 LTV < 80 Credit_Score < 620   
7  NaN      Mr. Cooper  DTI < 45 LTV < 80 Credit_Score < 620   

                                          Vesta Loan   QC  Note Pass/Fail   
5  https://beeline.beta.vestatech.io/loans/bad2d3...  NaN   NaN      Pass  \
6  https://beeline.beta.vestatech.io/loans/bad2d3...  NaN   NaN      Pass   
7  https://beeline.beta.vestatech.io/loans/bad2d3...  NaN   NaN      Pass   

    Rate    Pric